In [2]:
from rmgpy.species import Species
import rmgpy.molecule as molecule
import rmgpy.chemkin as chemkin
from rmgpy.data.rmg import RMGDatabase
from rmgpy.data.kinetics.library import LibraryReaction, KineticsLibrary
from rmgpy.data.kinetics.database import KineticsDatabase
import os, shutil, re, logging
from rmgpy.rmg.main import initializeLog, RMG
from rmgpy.chemkin import ChemkinWriter
from rmgpy.rmg.react import reactAll, react
import itertools

In [2]:
# This notebook is used to generate RO + NO = RONO = R + NO2 reactions (inc. ROO + NO = ROONO = RO + NO2)
# for an existing mechanism (existing RO's/ROO's)
# Used for low T combustion NOx cycle
# work on the speciesIndex branch

In [4]:
# extract species from xp1044 which is a dictionary made of xp1039, renamed to match xp1038

mechpath = '../../xp-C5NO/xp1044/new_dictionary.txt'

# read dictionary
originalSpeciesDict = chemkin.loadSpeciesDictionary(mechpath)

# ! renaming was already done below under title "Just rename species from RMG format to original names"

# read species with comments and change labels in the dictionary
#chem = open(mechpath+'chem.inp', 'r').readlines()
#lastline = ''
#readline = False
#for line in chem:
#    if 'SPECIES' in lastline:
#        readline = True
#    if 'END' in line:
#        readline = False
#    if readline:
#        spc, exmark, name = line.split()
#        name = name.split('(')[0]
#        originalSpeciesDict[spc].label=name
#    lastline = line

### Part 1 (RO + NO = RONO   //   ROO + NO = ROONO)

In [4]:
# save a sub-section of the dictionary

# save a dictionary file
basepath = '../../xp-C5NO/xp1044/'

#dic = 'ROO'
dic = 'RO'
#dic = 'all'


if dic == 'ROO':
    # save ROO species to dictionary
    ROO = []
    for label, spc in originalSpeciesDict.items():
        for atom1 in spc.molecule[0].vertices:
            if atom1.isOxygen():
                for atom2, bond12 in atom1.edges.items():
                    if atom2.isOxygen() and atom2.radicalElectrons == 1:
                        if spc.label != 'O2':
                            ROO.append(spc)
    chemkin.saveSpeciesDictionary(basepath+'species_dictionary_ROO.txt', ROO)


elif dic == 'RO':  # includes ROO
    # save RO species to dictionary
    RO = []
    for label, spc in originalSpeciesDict.items():
        for mol in spc.molecule:
            for atom1 in mol.vertices:
                if atom1.isOxygen() and atom1.radicalElectrons == 1:
                    #for atom2, bond12 in atom1.edges.items():
                        #if not atom2.isOxygen():
                    RO.append(spc)
    chemkin.saveSpeciesDictionary(basepath+'species_dictionary_RO.txt', RO)
    
    
elif dic == 'all':
    # save all species to dictionary
    species = []
    for label, spc in originalSpeciesDict.items():
        species.append(spc)
    chemkin.saveSpeciesDictionary(basepath+'species_dictionary_all.txt', species)

In [48]:
# run the generate reactions script with an input file composed of that sub-section dictionary (create input file using
# the generateInput script). This is xp1045 (was xp1041)

In [2]:
# Load only the RONO / ROONO species generated in xp1045 (was xp1041)
dic = '../../xp-C5NO/xp1045/chemkin/species_edge_dictionary.txt'
output = '../../xp-C5NO/xp1045/RONO.txt'

# read dictionary
speciesDict = chemkin.loadSpeciesDictionary(dic)

# save RONO species to dictionary
RONO = []
for label, spc in speciesDict.items():
    for atom1 in spc.molecule[0].vertices:
        if atom1.isOxygen() and atom1.radicalElectrons == 0:  # O
            for atom2, bond12 in atom1.edges.items():
                if atom2.isNitrogen() and bond12.isDouble() and atom2.lonePairs == 1:  # N=O
                    for atom3, bond23 in atom2.edges.items():
                        if atom3.isOxygen() and bond23.isSingle() and atom3.radicalElectrons == 0:  # RON=O/ROON=O
                            identical = False
                            for dicspc in RONO:
                                if dicspc.label.split('(')[1] == spc.label.split('(')[1]:
                                    identical = True
                            if not identical:
                                RONO.append(spc)
chemkin.saveSpeciesDictionary(output, RONO)


In [5]:
# Filter reactions in xp1045 (was xp1041),
# keeping only RO + NO = RONO (inc. ROO + NO = ROONO). This accomplishes 50% of the RONO cycle

mech = '../../xp-C5NO/xp1045/chemkin/chem_edge_annotated.inp'
dic = '../../xp-C5NO/xp1045/chemkin/species_edge_dictionary.txt'
output = '../../xp-C5NO/xp1045/'

speciesList, reactionList = chemkin.loadChemkinFile(path=mech, dictionaryPath=dic)

rxns = []
spcs = []
for rxn in reactionList:
    NO = False
    RO = False  # looking for both NO2 and RO in one side of the reaction
    RONO = False
    for spc in rxn.reactants + rxn.products:
        if spc.label == 'NO':  # check NO
            NO = True
        for mol in spc.molecule:  # check RO/ROO, inc. resonance structures
            for atom in mol.vertices:
                if atom.isOxygen() and atom.radicalElectrons == 1:
                    RO = True
        for mol in spc.molecule:  # check RONO/ROONO
            for atom1 in mol.vertices:
                if atom1.isOxygen() and atom1.radicalElectrons == 0:  # O
                    for atom2, bond12 in atom1.edges.items():
                        if atom2.isNitrogen() and bond12.isDouble() and atom2.lonePairs == 1:  # N=O
                            for atom3, bond23 in atom2.edges.items():
                                if atom3.isOxygen() and bond23.isSingle() and atom3.radicalElectrons == 0:  # RON=O/ROON=O
                                    RONO = True
    if NO and RO and RONO:
        rxns.append(rxn)
        for spc in rxn.reactants + rxn.products:
            if spc not in spcs:
                spcs.append(spc)
    
chemkin.saveChemkinFile(path=output+'RONO(1)output.inp', species=spcs, reactions=rxns, verbose = True, checkForDuplicates=False)
chemkin.saveSpeciesDictionary(output+'RONO(1)dic.txt', spcs)


### Part 2 (RONO = R + NO2   //   ROONO = RO + NO2)

In [ ]:
# run the generate reactions script with an input file composed of the RONO(1)dic.txt sub-section dictionary
# (create input file using the generateInput script). This is xp1046 (was xp1042)

In [11]:
# Filter reactions in xp1046 (was xp1042),
# keeping only RONO = R + NO2 (inc. ROONO = RO + NO2). This accomplishes 100% of the RONO cycle

mech = '../../xp-C5NO/xp1046/chemkin/chem_edge_annotated.inp'
dic = '../../xp-C5NO/xp1046/chemkin/species_edge_dictionary.txt'
output = '../../xp-C5NO/xp1046/'

speciesList, reactionList = chemkin.loadChemkinFile(path=mech, dictionaryPath=dic)

rxns = []
spcs = []
for rxn in reactionList:
    NO2 = False
    R = False  # looking for both NO2 and RO in one side of the reaction
    RONO = False
    for spc in rxn.reactants + rxn.products:
        if spc.label == 'NO2':  # check NO2
            NO2 = True
        for atom in spc.molecule[0].vertices:  # check R
            if atom.radicalElectrons == 1:
                R = True
        for mol in spc.molecule:  # check RONO/ROONO
            for atom1 in mol.vertices:
                if atom1.isOxygen() and atom1.radicalElectrons == 0:  # O
                    for atom2, bond12 in atom1.edges.items():
                        if atom2.isNitrogen() and bond12.isDouble() and atom2.lonePairs == 1:  # N=O
                            for atom3, bond23 in atom2.edges.items():
                                if atom3.isOxygen() and bond23.isSingle() and atom3.radicalElectrons == 0:  # RON=O/ROON=O
                                    RONO = True
    if NO2 and RO and RONO:
        rxns.append(rxn)
        for spc in rxn.reactants + rxn.products:
            if spc not in spcs:
                spcs.append(spc)
    
chemkin.saveChemkinFile(path=output+'RONO(2)output.inp', species=spcs, reactions=rxns, verbose = True, checkForDuplicates=False)
chemkin.saveSpeciesDictionary(output+'RONO(2)dic.txt', spcs)


### Part 3 (RH + NO2 = R + HONO)

### Just rename species from RMG format to original names

In [27]:
species = '../../xp-C5NO/xp1044/rename/species.inp'
dic = '../../xp-C5NO/xp1044/rename/dictionary.txt'
newdic = '../../xp-C5NO/xp1044/rename/new_dictionary.txt'

species = open(species,'r').readlines()
glossary = {}
for line in species:
    if 'SPECIES' in line or 'END' in line or line == '\n':
        continue
    name, mark, fullname = line.split()
    glossary[name] = fullname.split('(')[0]

dic = open(dic,'r').readlines()

lines = ''''''
for i in xrange(len(dic)):
    if '(' in dic[i]:
        lines += glossary[dic[i][0:-1]] + '\n'
    else:
        lines += dic[i]
        
with open(newdic,'w') as newdic:
        newdic.write(lines)

# code that was not used:

In [ ]:
# read dictionary
mechpath = 'xp1037_m/'
speciesDict = chemkin.loadSpeciesDictionary(mechpath+'species_dictionary.txt')

# functions
def createInputFile(theme, n, label, SMILES):
    n = str(n)
    if not os.path.exists('runs/'):
        os.mkdir('runs/')
    if not os.path.exists('runs/{0}-{1}/'.format(theme,n)):
        os.mkdir('runs/{0}-{1}/'.format(theme,n))
    inputFile = '''
database(thermoLibraries = ['primaryThermoLibrary'],reactionLibraries = [],
    seedMechanisms=[],kineticsDepositories=['training'],kineticsFamilies='default',kineticsEstimator='rate rules')
species(label="'''+str(label)+'''",reactive=True,structure=SMILES("'''+str(SMILES)+'''"))
species(label='NO',reactive=True,structure=SMILES('[N]=O'))
species(label='Ar',reactive=False,structure=adjacencyList("""1 Ar u0 p4 c0"""))
simpleReactor(temperature=(700,'K'),pressure=(1,'atm'),initialMoleFractions={"'''+str(label)+'''": 0.2,"NO": 0.2,"Ar": 0.6,},terminationTime=(0.001,'s'))
simulator(atol=1e-16,rtol=1e-8)
model(toleranceKeepInEdge=0,toleranceMoveToCore=0.05,toleranceInterruptSimulation=0.05,maximumEdgeSpecies=300000)
pressureDependence(method='modified strong collision',maximumGrainSize=(0.5,'kcal/mol'),minimumNumberOfGrains=250,temperatures=(300,1000,'K',10),pressures=(0.5,1.5,'bar',5),interpolation=('Chebyshev', 6, 4),maximumAtoms=16)
options(units='si',generateOutputHTML=False,generatePlots=False,saveEdgeSpecies=True,saveSimulationProfiles=False,saveRestartPeriod=None)
generatedSpeciesConstraints(allowed=['input species','seed mechanisms','reaction libraries'],maximumCarbonAtoms=10,maximumOxygenAtoms=10,maximumNitrogenAtoms=2,maximumSiliconAtoms=0,maximumSulfurAtoms=0,maximumHeavyAtoms=15,maximumRadicalElectrons=1,allowSingletO2=False)
'''
    path = 'runs/{0}-{1}/'.format(theme,n)
    with open(path+'input.py','w') as RMGInputFile:
        RMGInputFile.write(inputFile)
    return str(path)

# find ROO species
ROO = []
for label, spc in speciesDict.items():
    for atom1 in spc.molecule[0].vertices:
        if atom1.isOxygen():
            for atom2, bond12 in atom1.edges.items():
                if atom2.isOxygen() and atom2.radicalElectrons == 1:
                    if spc.label != 'O2(2)':
                        ROO.append(spc)
                        continue

In [ ]:
# generate ROO+NO rxns

n = 0
for spc in ROO:
    n += 1
    path = createInputFile('ROO', n, spc.label, spc.molecule[0].toSMILES())
    os.system('python generateReactions.py {0}input.py'.format(path)) 
    
print "Done."
    

In [ ]:
# merge models
# need to start with an empty chem.inp

n = 0
for spc in ROO:
    n += 1
    path = 'runs/ROO-{0}/chemkin/'.format(n)
    os.system('python mergeModels.py --model1 chem.inp species_dictionary.txt --model2 {0}chem_annotated.inp {0}species_dictionary.txt'.
            format(path))

print "Done."

In [ ]:
# read dictionary
path = 'xp1037_m/'
speciesDict = chemkin.loadSpeciesDictionary(path+'species_dictionary.txt')

# find ROO species
ROO = []
for label, spc in speciesDict.items():
    for atom1 in spc.molecule[0].vertices:
        if atom1.isOxygen():
            for atom2, bond12 in atom1.edges.items():
                if atom2.isOxygen() and atom2.radicalElectrons == 1:
                    if spc.label != 'O2(2)':
                        ROO.append(spc)
                        continue

In [ ]:
# generate ROO+NO rxns
NO = Species().fromSMILES('[N]=O')

database = RMGDatabase()
basePath = '/home/alongd/Code/RMG-database/input/'
database.load(
    path = basePath,
    thermoLibraries = ['primaryThermoLibrary'],
    transportLibraries = [],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsFamilies = 'default',
    kineticsDepositories = 'training',
    depository = False)
#for spc in ROO:
rxns = list(react([(NO),(ROO[0])]))

# run in loop, save in chem format once (w species, so provide thermo libraries)

In [ ]:
chem = []
for rxn in rxns:
    speciesList = []
    speciesList.append(rxn.reactants)
    speciesList.append(rxn.products)
    chem.append(chemkin.writeKineticsEntry(rxn,speciesList,verbose = False))
chem

In [ ]:

cwd = os.getcwd()
path = cwd+'/ROO/'
if not os.path.exists(path):
    os.mkdir(path)
level = logging.INFO
initializeLog(level, path+'RMG.log')
rmg = RMG(inputFile=path+'input.py', outputDirectory=path)
rmg.attach(ChemkinWriter(path))
rmg.initialize()
rmg.reactionModel.enlarge(reactEdge=True,
    unimolecularReact=rmg.unimolecularReact,
    bimolecularReact=rmg.bimolecularReact)
rmg.reactionModel.outputSpeciesList.extend(rmg.reactionModel.edge.species)
rmg.reactionModel.outputReactionList.extend(rmg.reactionModel.edge.reactions)
rmg.saveEverything()
rmg.finish()

In [ ]:
print('python $rmg/scripts/generateReactions.py {0}input.py'.format(path)) 

In [ ]:
# generate ROO+NO rxns
database = KineticsDatabase()
basePath = '/home/alongd/Code/RMG-database/input/kinetics/libraries/' # path to 'RMG-database/input/kinetics/libraries/'

level = logging.INFO

n = 0
for spc in ROO:
    n += 1
    path = createInputFile('ROO', n, spc.label, spc.molecule[0].toSMILES())
    initializeLog(level, path+'RMG.log')
    rmg = RMG(inputFile=path+'input.py', outputDirectory=path)
    rmg.attach(ChemkinWriter(path))
    rmg.initialize()
    rmg.reactionModel.enlarge(reactEdge=True,
        unimolecularReact=rmg.unimolecularReact,
        bimolecularReact=rmg.bimolecularReact)
    rmg.reactionModel.outputSpeciesList.extend(rmg.reactionModel.edge.species)
    rmg.reactionModel.outputReactionList.extend(rmg.reactionModel.edge.reactions)
    rmg.saveEverything()
    rmg.finish()
    
    
    